In [1]:
pip install tensorflow opencv-python scikit-learn matplotlib mediapipe

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


In [ ]:
import cv2 
 
import cv2 
import os 
 
# Create a folder for each gesture you want to recognize 
gestures = ['A', 'B', 'C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T,'U','V','W','X','Y','Z']


In [ ]:
# Create folders if they don't exist 

for gesture in gestures: 
    if not os.path.exists(f'dataset/{gesture}'): 
        os.makedirs(f'dataset/{gesture}') 

In [ ]:
# Capture images for each gesture 
cap = cv2.VideoCapture(0) 
 
for gesture in gestures: 
    print(f"Capturing images for {gesture}") 
    for img_num in range(100):  # Capture 100 images per gesture 
        ret, frame = cap.read() 
        if not ret: 
            break 
        cv2.putText(frame, f"Sign: {gesture}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2) 
        cv2.imshow("Frame", frame) 
        cv2.imwrite(f'dataset/{gesture}/{img_num}.jpg', frame) 
         
        if cv2.waitKey(1) & 0xFF == ord('q'): 
            break 
 
cap.release() 
cv2.destroyAllWindows() 
import cv2 
import numpy as np 
import os 

In [ ]:
# Load the dataset 
data = [] 
labels = [] 
for gesture in gestures: 
for img_name in os.listdir(f'dataset/{gesture}'): 
img = cv2.imread(f'dataset/{gesture}/{img_name}') 
img = cv2.resize(img, (64, 64))  # Resize to 64x64 
data.append(img) 
labels.append(gestures.index(gesture)) 

In [ ]:
# Convert to numpy arrays 
data = np.array(data) / 255.0  # Normalize pixel values to [0, 1] 
labels = np.array(labels) 
# Shuffle the data 
from sklearn.utils import shuffle 
data, labels = shuffle(data, labels, random_state=42) 
# Split into train and test sets 
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42) 
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense 

In [ ]:
# Define the CNN model 
model = Sequential() 
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3))) 
model.add(MaxPooling2D((2, 2))) 
model.add(Conv2D(64, (3, 3), activation='relu')) 
model.add(MaxPooling2D((2, 2))) 
model.add(Conv2D(128, (3, 3), activation='relu')) 
model.add(MaxPooling2D((2, 2))) 
model.add(Flatten()) 
model.add(Dense(128, activation='relu')) 
model.add(Dense(len(gestures), activation='softmax')) 
# Compile the model 
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) 
# Train the model 
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test)) 
# Evaluate the model 
test_loss, test_acc = model.evaluate(X_test, y_test) 
print(f"Test accuracy: {test_acc * 100:.2f}%") 

In [ ]:
# Predicting a new sign 
def predict_sign(image): 
img = cv2.resize(image, (64, 64)) 
img = np.expand_dims(img, axis=0) 
img = img / 255.0 
prediction = model.predict(img) 
return gestures[np.argmax(prediction)] 
cap = cv2.VideoCapture(0) 
while True: 
ret, frame = cap.read() 
if not ret: 
break 
# Make a prediction 
sign = predict_sign(frame) 

In [ ]:
# Display the prediction 
cv2.putText(frame, f"Predicted: {sign}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2) 
cv2.imshow("Frame", frame) 
if cv2.waitKey(1) & 0xFF == ord('q'): 
break 
cap.release() 
cv2.destroyAllWindows() 
import cv2 
import numpy as np 
from collections import deque 
# Initialize an empty list to store the predicted letters 
predicted_word = deque(maxlen=50)  # Using deque to store up to 50 characters 
# Function to predict sign from an image 
def predict_sign(image): 
img = cv2.resize(image, (64, 64))  # Resize to match the input size 
img = np.expand_dims(img, axis=0)  # Add an extra dimension (batch size) 
img = img / 255.0  # Normalize 
prediction = model.predict(img) 
return gestures[np.argmax(prediction)]  # Return the gesture with the highest probability 
cap = cv2.VideoCapture(0) 
while True: 
ret, frame = cap.read() 
if not ret: 
break 

In [ ]:
# Make a prediction for the current frame 
sign = predict_sign(frame) 
# Add the predicted sign to the list of letters forming the word 
if sign != "None":  # Assuming 'None' is returned when no valid gesture is detected 
predicted_word.append(sign) 
# Join the letters in the deque to form the current word 
current_word = ''.join(predicted_word) 
# Display the current word on the video feed 
cv2.putText(frame, f"Predicted: {current_word}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 
0, 0), 2) 
cv2.imshow("Frame", frame) 
# Press 'q' to quit the webcam 
if cv2.waitKey(1) & 0xFF == ord('q'): 
break 
cap.release() 
cv2.destroyAllWindows()